In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2, SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
# from pandas_profiling import ProfileReport

# Semana 1

## 1. Importação dos Dados
* Dados importados a partir de um arquivo CSV e armazenados em uma estrutura Dataframe do Pandas

In [9]:
df_dados = pd.read_csv('Banco_de_dados_bancosaudemental.csv',encoding="ISO-8859-1",delimiter=';')

In [11]:
df_dados.head()

,SAMPLEID,SC1,SC1_1,SC2,SC2A,SC3,SC3A,SC7,SC8_1,SC8_2,...,notreat12_notsevere,notreat12_anypredisposing,notreat12_service,qids_none,qids_mild,qids_mod,qids_sev,qids_vsev,qids_sevverysev,_merge
0,3201000001-01,32,1,4,4.0,5,1.0,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
1,3201000002-01,31,1,8,4.0,5,1.0,3,3,3,...,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
2,3201000003-01,60,1,9,4.0,1,NaN,2,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,matched (3)
3,3201000004-01,34,1,5,4.0,5,1.0,3,3,2,...,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
4,3201000005-01,84,2,41,4.0,4,5.0,3,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)


## 2. Selecionando os Atributos Relacionados à Depressão
* Na base de dados do São Paulo Megacity os Atributos relacionados a Depressão começam com a letra "D", com 3 exceções que são tratadas no código a seguir, tais atributos são selecionados e armazenados em uma lista

In [12]:
lista=[]
for x in df_dados.columns:
  if x[0] == "D":
    lista.append(x)
    
    
#Marcadores de Diagnóstico

# Major Depressive Disorder (MDD):
lista.extend(['dsm_mddh']) #'d_mddh12','d_mddh30'

# Major Depressive Episode (MDE):
#lista.extend(['d_mde12', 'd_mde30','dsm_mde'])


# Minor Depressive Disorder (MND):
#lista.extend(['d_mndh12', 'd_mndh30', 'dsm_mndh','d_mnd12','d_mnd30','dsm_mnd'])

#Recurrent Brief Depression Disorder (RBD):
#lista.append('dsm_rbd')


#Removendo Atributos iniciados com "D" não relacionados a Depressão:
lista = [x for x in lista if not x == 'D26_YESC' or x == 'D0' or x == 'D29C1']

## 3. Criando novo Dataframe a partir dos Atributos Selecionados
* Utilizando a lista de atributos relacionados à depressão é criado um novo dataframe somente com tais variáveis

In [5]:
df_depression = df_dados[lista]

In [6]:
df_depression

,D0,D1,D1A,D1B,D2,D9,D9A1,D9A2,D10,D11,...,D84A,D84B,D84C,D86,D87,D87A,D87_1,D88,D89,dsm_mddh
0,1.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.0,NaN,5
1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,5
2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.0,1.0,NaN,1.0,1.0,59.0,2.0,4.0,NaN,1
3,1.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,NaN,5
4,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.0,NaN,5
5033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [7]:
# REMOVENDO PONTOS DE VERIFICAÇÃO/INSTRUÇÃO DO ENTREVISTADOR
df_depression.drop(['D10','D11','D13','D20','D25','D27','D27A','D29D','D37E','D37G','D37L','D38_1','D38A_2','D38A_5','D38A_8','D38B_1','D40','D44','D49','D55','D57','D60','D62_1','D62_2','D67','D88','D89'],axis=1,inplace=True)


/home/gabriel/.local/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
df_depression

,D0,D1,D1A,D1B,D2,D9,D9A1,D9A2,D12,D12A,...,D72A,D84,D84A,D84B,D84C,D86,D87,D87A,D87_1,dsm_mddh
0,1.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5
1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5
2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,59.0,1.0,59.0,1.0,NaN,1.0,1.0,59.0,2.0,1
3,1.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5
4,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,5.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5
5033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


## 4. Análise e Tratamento de Valores Nulos
* Colunas com alto indíce de valores nulos serão retiradas nesta etapa

In [9]:
excluir_nulos = []
nulos = df_depression.isnull().sum()
nulos.sort_values(ascending=False,inplace=True)
for i in nulos.index:
    print(i,"--->",nulos[i])
    if nulos[i] >= 5037*0.8:
        excluir_nulos.append(i)
        

D23A13 ---> 5037
D23A10 ---> 5037
D23A12 ---> 5037
D23A14 ---> 5037
D23A15 ---> 5037
D23A09 ---> 5037
D23A16 ---> 5037
D23A17 ---> 5037
D23A18 ---> 5037
D23A11 ---> 5037
D23A07 ---> 5036
D23A08 ---> 5036
D50 ---> 5035
D45 ---> 5034
D48 ---> 5033
D47 ---> 5033
D23A06 ---> 5033
D42 ---> 5031
D46 ---> 5031
D43 ---> 5031
D42_1 ---> 5031
D23A05 ---> 5030
D41 ---> 5026
D23A04 ---> 5010
D9A2 ---> 5008
D9A1 ---> 5007
D9 ---> 4992
D37B ---> 4973
D23A03 ---> 4959
D38C ---> 4959
D87A ---> 4945
D38A_9 ---> 4939
D15 ---> 4939
D29C1 ---> 4925
D58 ---> 4887
D38A_7B ---> 4881
D61 ---> 4877
D59 ---> 4863
D59A ---> 4863
D22D1 ---> 4859
D84C ---> 4849
D22C ---> 4837
D22D ---> 4837
D38A_7A ---> 4835
D14A ---> 4828
D38A_6 ---> 4825
D26D1 ---> 4819
D26D ---> 4818
D26O ---> 4808
D29B ---> 4805
D23A02 ---> 4803
D39B ---> 4776
D56 ---> 4756
D37M_2 ---> 4751
D37N_2 ---> 4750
D26H ---> 4745
D54 ---> 4741
D37F_3 ---> 4739
D37F_2 ---> 4739
D37F_1 ---> 4739
D84A ---> 4739
D39A ---> 4739
D84B ---> 4739
D26EE ---> 47

* ###  Exclusão das colunas com 80% ou mais de valores nulos

In [10]:
df_depression_c_nulos = df_depression

In [11]:
df_depression.drop(excluir_nulos,axis=1,inplace=True)

/home/gabriel/.local/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
variaveis_count = df_depression.count().sort_values(ascending = False)

In [13]:
lista = []
for i in range(len(df_depression)):
    x = (len(df_depression.iloc[i].dropna())/len(variaveis_count))*100
    if x >= 50:  # Porcentagem das colunas sem valor nulo para cada entrada
        lista.append(True)
    else:
        lista.append(False)

* ### (CANCELADO) Serão utilizadas somente linhas com 65% ou mais de variáveis não nulas (CANCELADO)

In [14]:
# "df_depression_65" contém as entradas do "df_depression" com 65% das colunas sem valor nulo
#df_depression_65 = df_depression.iloc[lista]

In [15]:
df_depression_65 = df_depression

## 5. Pandas Profiling (df_depression_65)
* Geração de relatório 

In [16]:
#pfl_df_depression65 = ProfileReport(df_depression_65, title='Pandas Profiling Report', html={'style':{'full_width':False}},minimal=True)
#pfl_df_depression65.to_file(output_file="pfl_df_depression65.html")

## 6. Métricas e Correlações
* Tabela de correlação e função describe() do Dataframe

In [17]:
def verifica_maiores_correlacoes_absolutas(dados,correlacao=0.80):
    #objeto recebido e do tipo pandas.dataframe()
    #correlacao deve ser do tipo float
    #padrao verificado para correlacoes e acima de 80%
    if correlacao>=1:
        correlacao=(correlacao/100.0)
    corr = dados.corr()
    lista = []
    for coluna in corr.columns:
        for linha in corr.index:
            if (coluna != linha) and (abs(corr[coluna][linha]) >= correlacao):
                #print(coluna,' possui com  ',linha,' Correlacao de -->',corr[coluna][linha])
                dict = {'coluna':coluna,'linha':linha,'corr':corr[coluna][linha]}
                lista.append(dict)
                #print("%s  Vs   %s \t\t || Correlacao de --> %f ||"%(coluna,linha,corr[coluna][linha]))
    return lista

In [18]:
df_depression_65.describe()

,D0,D1,D1A,D12,D12A,D16,D17,D18,D19,D21,...,D26II,D28,D28A,D29,D29A,D37D,D38,D72,D87_1,dsm_mddh
count,3390.000000,2919.000000,2479.000000,3345.000000,1655.000000,1705.000000,1599.000000,1599.000000,1599.000000,1527.000000,...,1212.000000,1209.000000,1040.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,3390.000000,5037.000000
mean,1.152212,1.609113,2.407019,2.982661,27.093051,3.228152,2.806129,2.058161,2.497811,1.759659,...,2.154290,3.057072,2.181731,90.851944,4.247312,2.659222,2.973532,3.395368,105.194985,4.297995
std,0.394510,1.452350,1.919308,2.006493,149.751490,1.214701,1.022249,1.131413,1.190740,1.600048,...,1.825172,1.290266,1.109350,282.879878,1.597041,1.131868,2.029002,1.968084,305.401433,1.521732
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,5.000000,2.000000,1.000000,1.000000,0.000000,5.000000
50%,1.000000,1.000000,1.000000,1.000000,2.000000,4.000000,3.000000,2.000000,2.000000,1.000000,...,1.000000,3.000000,2.000000,2.000000,5.000000,3.000000,1.000000,5.000000,0.000000,5.000000
75%,1.000000,1.000000,5.000000,5.000000,5.000000,4.000000,4.000000,3.000000,4.000000,1.000000,...,5.000000,4.000000,3.000000,3.000000,5.000000,3.000000,5.000000,5.000000,1.000000,5.000000
max,3.000000,8.000000,8.000000,8.000000,999.000000,8.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,5.000000,8.000000,999.000000,8.000000,9.000000,9.000000,9.000000,999.000000,5.000000


In [19]:
lista_corr = verifica_maiores_correlacoes_absolutas(df_depression_65,correlacao=0.3)

In [20]:
lista_corr_2 = []
for i in lista_corr:
    if i['coluna'] == "dsm_mddh":
        lista_corr_2.append(i)
        
        
        
for i in lista_corr_2:
    print("%s  Vs   %s \t\t || Correlacao de --> %f ||"%(i['coluna'],i['linha'],i['corr']))
    print('')


dsm_mddh  Vs   D1A 		 || Correlacao de --> 0.317456 ||

dsm_mddh  Vs   D12 		 || Correlacao de --> 0.527337 ||

dsm_mddh  Vs   D16 		 || Correlacao de --> -0.348808 ||

dsm_mddh  Vs   D19 		 || Correlacao de --> 0.316013 ||

dsm_mddh  Vs   D21 		 || Correlacao de --> 0.556864 ||

dsm_mddh  Vs   D26J 		 || Correlacao de --> 0.320987 ||

dsm_mddh  Vs   D26R 		 || Correlacao de --> 0.340269 ||



## 7. Carregamento do Dicionário de Variáveis
* O carregamento é feito a partir de um arquivo CSV e armazenado em uma estrutura Dataframe do pandas, logo após é impresso as definições de cada variável

In [21]:
df_variaveis_depre_temp = pd.read_csv('CIDI Depressão - Mapeamento de variáveis _2_CSV.csv',delimiter=';',encoding='ISO-8859-1')

In [22]:
df_variaveis_depre_temp = df_variaveis_depre_temp.rename(columns={'Pergunta/Instrução':'Pergunta_Instrucao','Página':'Pagina','Variável na base de dados': 'Variavel','Possíveis valores':'Possiveis_valores'})

In [23]:
df_variaveis_depre_temp = df_variaveis_depre_temp[(df_variaveis_depre_temp.Variavel != 'Not Found') & (df_variaveis_depre_temp.Variavel.notna())]

In [24]:
Variaveis_D = df_variaveis_depre_temp['Variavel'].str.split(',')

In [25]:
df_variaveis_depre_v2 = pd.DataFrame(columns=df_variaveis_depre_temp.columns)

In [26]:
df_variaveis_depre_v2 = df_variaveis_depre_v2[['Variavel','Pergunta_Instrucao', 'Possiveis_valores']]

In [27]:
for i in Variaveis_D.index:
    for variavel in Variaveis_D[i]:
        df_variaveis_depre_v2.loc[len(df_variaveis_depre_v2)] = [variavel.strip(),df_variaveis_depre_temp.loc[i].Pergunta_Instrucao,df_variaveis_depre_temp.loc[i].Possiveis_valores]

In [28]:
dict_variaveis_depre_v2 = {}
for row in df_variaveis_depre_v2.itertuples():
    dict_variaveis_depre_v2[row.Variavel] = row.Pergunta_Instrucao

In [29]:
dict_valores_depre_v2 = {}
for row in df_variaveis_depre_v2.itertuples():
    dict_valores_depre_v2[row.Variavel] = row.Possiveis_valores

In [30]:
for i in dict_variaveis_depre_v2:
    print('%s ---> %s\n\n'%(i,dict_variaveis_depre_v2[i]))

D1 ---> O(A) Sr(a). mencionou anteriormente que teve períodos de vários dias ou mais em que se sentia triste, vazio(a) oudeprimido(a) a maior parte do dia. Durante fases como essa, o(a) Sr(a). alguma vez se sentiu 
desanimado(a) em relação a como as coisas andavam em sua vida?


D1A ---> Durante as fase sem que se sentia triste, vazio(a) ou deprimido(a), o(a) Sr(a). alguma vez perdeu o interessena maioria dascoisas como trabalho, diversão e outras atividades de que habitualmente gostava?


D1B ---> Durante as fases em que se sentia triste, vazio(a) ou deprimido(a), o(a) Sr(a). alguma vez perdeu o interessena maioria das coisas como trabalho, diversão e outras atividades de que habitualmente gostava?


D2 ---> (O Sr/a Sra.) mencionou anteriormente queteve períodos de vários dias ou mais em que se sentia desanimado(a)em relação a como as coisas andavam em sua vida. Durante fases como essa, o(a) Sr(a). alguma vez 
perdeu o interesse na maioria das coisas como trabalho, diversão e outras a

# Semana 2

## 1. Balanceamento dos Dados
* Os dados são balanceados a partir da variável dependente, são igualados os indivíduos com diagnóstico positivo e negativo para depressão

In [31]:
count_5 = len(df_depression_65[df_depression_65.dsm_mddh == 5]) # Diagnóstico Positivo
count_5

4153

In [32]:
count_1 = len(df_depression_65[df_depression_65.dsm_mddh == 1]) # Diagnóstico Negativo
count_1

884

In [33]:
df_dep_65_bal = df_depression_65[df_depression_65.dsm_mddh == 1].sample(count_5,replace=True,random_state = 5)
df_dep_65_bal = pd.concat([df_dep_65_bal,df_depression_65[df_depression_65.dsm_mddh == 5]],axis=0)

In [34]:
# Dataframe Balanceado
df_dep_65_bal.head()

,D0,D1,D1A,D12,D12A,D16,D17,D18,D19,D21,...,D26II,D28,D28A,D29,D29A,D37D,D38,D72,D87_1,dsm_mddh
4962,1.0,5.0,NaN,1.0,NaN,4.0,2.0,2.0,2.0,1.0,...,1.0,2.0,2.0,1.0,5.0,2.0,5.0,5.0,0.0,1
1207,1.0,1.0,5.0,1.0,NaN,8.0,4.0,1.0,1.0,1.0,...,1.0,5.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1
4072,1.0,1.0,1.0,1.0,NaN,4.0,2.0,3.0,2.0,1.0,...,5.0,4.0,1.0,998.0,1.0,1.0,1.0,1.0,4.0,1
714,1.0,1.0,1.0,1.0,NaN,3.0,4.0,1.0,1.0,1.0,...,1.0,4.0,1.0,2.0,5.0,2.0,1.0,1.0,0.0,1
2373,1.0,1.0,1.0,5.0,3.0,NaN,1.0,3.0,2.0,1.0,...,1.0,2.0,1.0,4.0,5.0,1.0,1.0,1.0,0.0,1


## 2. Definindo Conjuntos de Treino e Teste
* O dataframe será dividido em um conjunto de treino e outro conjunto de teste. O conjunto de treino possui 80% do volume das linhas enquanto o conjunto de teste possui os 20% restantes.

In [35]:
X_train,X_test,y_train,y_test = train_test_split(df_dep_65_bal,df_dep_65_bal['dsm_mddh'],train_size = 0.8,test_size = 0.2,random_state = 5)

In [36]:
print(len(X_train))
print(len(X_test))

6644
1662


## 3. Selecionando Principais Atributos com Chi2
* O teste do chi2 irá selecionar os N melhores atributos para serem utilizados no modelo estatístico

In [37]:
X_train.fillna(0,inplace=True)
Melhores_Atributos = SelectKBest(chi2,k=50).fit(X_train.drop('dsm_mddh',axis=1),y_train)
Lista_Melhores = list(X_train.drop('dsm_mddh',axis=1).columns[Melhores_Atributos.get_support()])

/home/gabriel/.local/lib/python3.7/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [38]:
X_train.drop('dsm_mddh',axis=1).columns[Melhores_Atributos.get_support()]

Index(['D0', 'D1', 'D1A', 'D12', 'D12A', 'D16', 'D17', 'D18', 'D19', 'D21',
       'D22', 'D22A', 'D22B', 'D22B1', 'D23', 'D24A', 'D24B', 'D24C', 'D24D',
       'D24E', 'D24F', 'D26A', 'D26G', 'D26I', 'D26J', 'D26L', 'D26P', 'D26R',
       'D26S', 'D26T', 'D26U', 'D26W', 'D26X', 'D26Y', 'D26Z', 'D26AA',
       'D26BB', 'D26CC', 'D26FF', 'D26GG', 'D26HH', 'D26II', 'D28', 'D28A',
       'D29', 'D29A', 'D37D', 'D38', 'D72', 'D87_1'],
      dtype='object')

In [39]:
Score_Melhores = pd.Series(Melhores_Atributos.scores_)
Score_Melhores.index = X_train.drop('dsm_mddh',axis=1).columns
Score_Melhores = Score_Melhores.sort_values(ascending=False)
lst_melhores = []
#for i in Score_Melhores.index:
    #if Score_Melhores[i] > 5:
        #print('%s --->  %f'%(i,Score_Melhores[i]))
    #lst_melhores.append(i)

In [40]:
for i in Score_Melhores.index:
    print('%s --->  %f'%(i,Score_Melhores[i]))

D29 --->  245091.046048
D22A --->  71613.648074
D12A --->  26060.173044
D29A --->  11264.917591
D26CC --->  9739.421463
D26Z --->  9726.799012
D26I --->  9158.785253
D28 --->  8632.328709
D26W --->  8456.142643
D72 --->  8271.317503
D22B1 --->  8033.489477
D38 --->  7691.447261
D26BB --->  7198.903276
D26FF --->  6813.667643
D37D --->  6790.707648
D16 --->  6728.172891
D26L --->  6506.282776
D17 --->  6475.314462
D26T --->  6091.230501
D26AA --->  5980.828100
D26U --->  5828.564284
D26X --->  5684.878095
D26P --->  5482.885995
D26S --->  5451.992217
D26A --->  5376.961717
D24D --->  5248.628509
D28A --->  5067.465889
D26GG --->  4907.520752
D26II --->  4873.614182
D23 --->  4731.333909
D24F --->  4529.564787
D26Y --->  4331.833051
D24B --->  4302.096028
D26G --->  4205.152263
D26J --->  4127.138550
D24E --->  3980.614238
D26HH --->  3945.167004
D22 --->  3544.781779
D26R --->  3521.679239
D19 --->  3401.984105
D24C --->  3009.135961
D24A --->  2794.292461
D18 --->  2733.365029
D87_1 --

In [41]:
lst_melhores = Lista_Melhores

In [42]:
for i in dict_variaveis_depre_v2:
    if i in lst_melhores:
        print('%s ---> %s\n\n'%(i,dict_variaveis_depre_v2[i]))

D1 ---> O(A) Sr(a). mencionou anteriormente que teve períodos de vários dias ou mais em que se sentia triste, vazio(a) oudeprimido(a) a maior parte do dia. Durante fases como essa, o(a) Sr(a). alguma vez se sentiu 
desanimado(a) em relação a como as coisas andavam em sua vida?


D1A ---> Durante as fase sem que se sentia triste, vazio(a) ou deprimido(a), o(a) Sr(a). alguma vez perdeu o interessena maioria dascoisas como trabalho, diversão e outras atividades de que habitualmente gostava?


D12 ---> Alguma vez o(a) Sr(a). teve um período que tenha durado duas semanas ou mais em que se sentiu (FRASE-CHAVE) a maior parte do dia, quase todos os dias?


D12A ---> Quantos dias durou o período mais longo em que o(a) Sr(a). se sentiu (FRASE-CHAVE) durante a maior parte do dia?


D16 ---> Pense nas fases de duas semanas ou mais em que (esse problema/esses problemas) com seu estado de ânimo(foi/foram) mais intenso(s) e freqüente(s). Nessas fases, o(a) Sr(a). geralmente se sentia (FRASE-CHAVE) 
d

In [43]:
lst_melhores.append('dsm_mddh')
X_train = X_train[lst_melhores]
X_test = X_test[lst_melhores]

In [44]:
len(lst_melhores)

51

# Semana 3
# Premissas da Regressão Logística

## 1. Relação Linear entre Variável Dependente e Variáveis Independentes

* Uma relação linear é uma tendência nos dados que pode ser modelada por uma linha reta. Por exemplo, suponha que uma empresa aérea deseja estimar o impacto do preço dos combustíveis sobre o custo dos voos. Eles determinam que, para cada aumento de um dólar no preço de um galão de combustível, o custo do voo Los Angeles-New York aumenta em torno de US$ 3500. Isso descreve uma relação linear entre o custo do combustível de aviação e o custo do voo.


In [45]:
for i in lst_melhores:
    print(X_train[['dsm_mddh',i]].corr())
    print('')

          dsm_mddh        D0
dsm_mddh  1.000000 -0.307605
D0       -0.307605  1.000000

          dsm_mddh        D1
dsm_mddh  1.000000 -0.085528
D1       -0.085528  1.000000

          dsm_mddh       D1A
dsm_mddh  1.000000 -0.103882
D1A      -0.103882  1.000000

          dsm_mddh       D12
dsm_mddh  1.000000  0.255066
D12       0.255066  1.000000

          dsm_mddh      D12A
dsm_mddh  1.000000  0.068952
D12A      0.068952  1.000000

          dsm_mddh       D16
dsm_mddh  1.000000 -0.757858
D16      -0.757858  1.000000

          dsm_mddh       D17
dsm_mddh  1.000000 -0.802012
D17      -0.802012  1.000000

          dsm_mddh       D18
dsm_mddh  1.000000 -0.567144
D18      -0.567144  1.000000

          dsm_mddh       D19
dsm_mddh  1.000000 -0.585832
D19      -0.585832  1.000000

          dsm_mddh       D21
dsm_mddh  1.000000 -0.290589
D21      -0.290589  1.000000

          dsm_mddh       D22
dsm_mddh  1.000000 -0.503899
D22      -0.503899  1.000000

          dsm_mddh      D22A
dsm

In [46]:
#X_train.drop(['D22C', 'D64A', 'D87_1', 'D64C', 'D59', 'D39A', 'D38A_1','D29' ,'D38B','D59A','D68', 'D54', 'D37H_1'],axis=1,inplace=True)
#X_test.drop(['D22C', 'D64A', 'D87_1', 'D64C', 'D59', 'D39A', 'D38A_1','D29' ,'D38B','D59A','D68', 'D54', 'D37H_1'],axis=1,inplace=True)

In [47]:
#for i in ['D22C', 'D64A', 'D87_1', 'D64C', 'D59', 'D39A', 'D38A_1','D29' ,'D38B','D59A','D68', 'D54', 'D37H_1']:
#    lst_melhores.remove(i)

In [48]:
'''lst_exclusao = []
for i in lst_melhores:
    corr_dsm = X_train[['dsm_mddh',i]].corr()
    print(corr_dsm[i][0])'''

"lst_exclusao = []\nfor i in lst_melhores:\n    corr_dsm = X_train[['dsm_mddh',i]].corr()\n    print(corr_dsm[i][0])"

### Observações:
* As Variáveis "D64A", "D64C", "D84A", "D59", "D39A", "D38A_1", "D29", "D38B", "D59A","D68", "D54" e "D37H_1" possuem Relação Linear (Pearson) abaixo/igual a 0.2 com a Variável Dependente, portanto não serão utilizadas no Modelo

* A Variável "D22C" não será utilizada por estar nula.

## 2. Ausência de Heterocedasticidade

* Fenômeno onde a variância dos erros de um modelo matemático não é igual para cada observação

## 3. Ausência de Multicolinearidade

* Multicolinearidade em regressão é uma condição que ocorre quando algumas variáveis preditoras no modelo estão correlacionadas a outras variáveis preditoras. A multicolinearidade forte é problemática porque pode aumentar a variância dos coeficientes de regressão, tornando-os instáveis.


In [49]:
X_train.drop(['dsm_mddh'],axis=1).corr()

,D0,D1,D1A,D12,D12A,D16,D17,D18,D19,D21,...,D26HH,D26II,D28,D28A,D29,D29A,D37D,D38,D72,D87_1
D0,1.000000,0.198367,0.178367,0.451710,0.054883,0.340831,0.322440,0.322451,0.330139,0.243603,...,0.207335,0.252753,0.269152,0.244672,0.074445,0.296265,0.291667,0.247214,0.270141,0.114492
D1,0.198367,1.000000,0.019737,0.325418,0.021804,0.122706,0.132153,0.143894,0.150147,0.129296,...,0.066391,0.058421,0.071271,0.086402,0.008595,0.104943,0.078558,0.082278,0.082250,0.083195
D1A,0.178367,0.019737,1.000000,0.257399,0.053842,0.123893,0.134451,0.165300,0.186210,0.153147,...,0.125906,0.100251,0.066622,0.112721,-0.025606,0.098777,0.105975,0.097407,0.118816,0.071923
D12,0.451710,0.325418,0.257399,1.000000,0.162789,-0.404344,-0.285214,-0.224286,-0.226122,-0.205610,...,-0.187048,-0.169072,-0.244214,-0.192700,-0.074465,-0.243042,-0.233602,-0.193522,-0.204779,0.070590
D12A,0.054883,0.021804,0.053842,0.162789,1.000000,-0.091158,-0.070317,-0.063297,-0.056875,-0.052911,...,-0.049245,-0.050178,-0.061342,-0.047272,-0.018899,-0.064157,-0.065399,-0.054934,-0.058490,0.054248
D16,0.340831,0.122706,0.123893,-0.404344,-0.091158,1.000000,0.832958,0.620591,0.648450,0.531086,...,0.518817,0.488745,0.727547,0.553194,0.182292,0.724299,0.701013,0.578794,0.602034,0.058299
D17,0.322440,0.132153,0.134451,-0.285214,-0.070317,0.832958,1.000000,0.595965,0.636004,0.547004,...,0.489986,0.472824,0.788997,0.599814,0.165626,0.741506,0.736398,0.607609,0.610590,0.042760
D18,0.322451,0.143894,0.165300,-0.224286,-0.063297,0.620591,0.595965,1.000000,0.804297,0.556702,...,0.402352,0.442575,0.472779,0.490318,0.136217,0.569178,0.544328,0.442362,0.522747,0.032435
D19,0.330139,0.150147,0.186210,-0.226122,-0.056875,0.648450,0.636004,0.804297,1.000000,0.585960,...,0.426497,0.445424,0.469171,0.545397,0.128576,0.582138,0.565170,0.450797,0.530530,0.032660
D21,0.243603,0.129296,0.153147,-0.205610,-0.052911,0.531086,0.547004,0.556702,0.585960,1.000000,...,0.190751,0.194485,0.270039,0.225434,0.066665,0.275195,0.269793,0.220632,0.235287,0.041194


* ####  D61  Vs. D56
* #### D72A Vs. D87A
* #### D72A Vs. D84C
* #### D87A Vs. D84C
* #### D26F Vs. D84C
* #### D37I_1 Vs. D84C

In [50]:
#X_train.drop(['D84C','D61','D87A'],axis=1,inplace=True)
#X_test.drop(['D84C','D61','D87A'],axis=1,inplace=True)

## 4. Resíduos

* Os resíduos indicam a variação natural dos dados, um fator aleatório (ou não) que o modelo não capturou. Se as pressuposições do modelo são violadas, a análise será levada a resultados duvidosos e não confiáveis para inferência. Essas falhas do modelo nos pressupostos podem ser oriundas de diversos fatores como não linearidade, não-normalidade, heterocedasticidade, não-independência e isso pode ser causado por pontos atípicos (observações discrepantes), que podem influenciar, ou não, no ajuste do modelo.


## 5. Regressão Logística

In [51]:
X_train.fillna(0,inplace=True)

In [52]:
X_test.fillna(0,inplace=True)

In [53]:
modelo = LogisticRegression(random_state=5,max_iter=1000000,C=3).fit(X_train.drop(['dsm_mddh'],axis=1),y_train)

In [54]:
score_treino = modelo.score(X_train.drop(['dsm_mddh'],axis=1),y_train)

In [55]:
print('Score de Acuracia [TREINO]:',score_treino)

Score de Acuracia [TREINO]: 0.964780252859723


In [56]:
score_teste = modelo.score(X_test.drop(['dsm_mddh'],axis=1),y_test)

In [57]:
print('Score de Acuracia [TESTE]:',score_teste)

Score de Acuracia [TESTE]: 0.9602888086642599


## Semana 4

## 1.  Avaliação do Modelo

### 1.1 Matriz de Confusão
* Matriz que permite visualizar o desempenho do modelo

In [58]:
y_pred = modelo.predict(X_test.drop(['dsm_mddh'],axis=1))
matriz_confusao = confusion_matrix(y_test,y_pred)
print(matriz_confusao)

[[817  14]
 [ 52 779]]


In [59]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.98      0.96       831
           5       0.98      0.94      0.96       831

    accuracy                           0.96      1662
   macro avg       0.96      0.96      0.96      1662
weighted avg       0.96      0.96      0.96      1662



![Precision](https://i.imgur.com/lBX6xb0.png)
![Recall](https://i.imgur.com/auzyA0v.png)
![F1-Score](https://i.imgur.com/uRnhSGP.png)


* #### f1-score = Média harmônica de Precision e Recall

##### NULOS 50 BEST
D12A ---> 486
D22B1 ---> 117
D22B ---> 99
D22A ---> 99
D24D ---> 72
D26I ---> 47
D1A ---> 46
D16 ---> 39
D24B ---> 31
D1 ---> 19
D27A ---> 13
D28 ---> 2
D29 ---> 2
D29A ---> 2
D29D ---> 2
D37E ---> 2
D37G ---> 2
D38 ---> 2
D72 ---> 2
D26A ---> 0
D24F ---> 0
D24E ---> 0
D24C ---> 0
D18 ---> 0
D24A ---> 0
D23 ---> 0
D22 ---> 0
D19 ---> 0
D17 ---> 0
D26G ---> 0
dsm_mddh ---> 0
D26J ---> 0
D26L ---> 0
D87_1 ---> 0
D26II ---> 0
D26HH ---> 0
D26GG ---> 0
D26FF ---> 0
D26CC ---> 0
D26BB ---> 0
D26AA ---> 0
D26Z ---> 0
D26Y ---> 0
D26X ---> 0
D26W ---> 0
D26U ---> 0
D26T ---> 0
D88 ---> 0
D26R ---> 0
D26P ---> 0
D26S ---> 0


In [60]:
#BALANCEANDO O DATAFRAME
count_5_2 = len(df_depression[df_depression.dsm_mddh == 5]) # Diagnóstico Positivo
count_5_2

4153

In [61]:
count_1_2 = len(df_depression[df_depression.dsm_mddh == 1]) # Diagnóstico Negativo
count_1_2

884

In [62]:
df_dep_bal = df_depression[df_depression.dsm_mddh == 1].sample(count_5_2,replace=True,random_state = 5)
df_dep_bal = pd.concat([df_dep_bal,df_depression[df_depression.dsm_mddh == 5]],axis=0)

In [63]:
#CONJUNTO DE TREINO E TESTE
X_train2,X_test2,y_train2,y_test2 = train_test_split(df_dep_bal,df_dep_bal['dsm_mddh'],train_size = 0.8,test_size = 0.2,random_state = 5)

In [64]:
X_train2.fillna(0,inplace=True)
Melhores_Atributos2 = SelectKBest(chi2,k=50).fit(X_train2.drop('dsm_mddh',axis=1),y_train2)
Lista_Melhores2 = list(X_train2.drop('dsm_mddh',axis=1).columns[Melhores_Atributos2.get_support()])

/home/gabriel/.local/lib/python3.7/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [65]:
X_train2.drop('dsm_mddh',axis=1).columns[Melhores_Atributos2.get_support()]

Index(['D0', 'D1', 'D1A', 'D12', 'D12A', 'D16', 'D17', 'D18', 'D19', 'D21',
       'D22', 'D22A', 'D22B', 'D22B1', 'D23', 'D24A', 'D24B', 'D24C', 'D24D',
       'D24E', 'D24F', 'D26A', 'D26G', 'D26I', 'D26J', 'D26L', 'D26P', 'D26R',
       'D26S', 'D26T', 'D26U', 'D26W', 'D26X', 'D26Y', 'D26Z', 'D26AA',
       'D26BB', 'D26CC', 'D26FF', 'D26GG', 'D26HH', 'D26II', 'D28', 'D28A',
       'D29', 'D29A', 'D37D', 'D38', 'D72', 'D87_1'],
      dtype='object')

In [66]:
Lista_Melhores2.append('dsm_mddh')
X_train2_full = X_train2
X_test2_full = X_test2
X_train2 = X_train2[Lista_Melhores2]
X_test2 = X_test2[Lista_Melhores2]

In [67]:
X_train2.fillna(0,inplace=True)
X_test2.fillna(0,inplace=True)

In [68]:
modelo2 = LogisticRegression(random_state=5,max_iter=1000000,C=3).fit(X_train2.drop(['dsm_mddh'],axis=1),y_train2)

In [69]:
score_treino2 = modelo2.score(X_train2.drop(['dsm_mddh'],axis=1),y_train2)

In [70]:
print('Score de Acuracia [TREINO]:',score_treino2)

Score de Acuracia [TREINO]: 0.964780252859723


In [71]:
score_teste2 = modelo2.score(X_test2.drop(['dsm_mddh'],axis=1),y_test2)

In [72]:
print('Score de Acuracia [TESTE]:',score_teste2)

Score de Acuracia [TESTE]: 0.9602888086642599


In [73]:
X_train2_full = X_train2_full[lst_melhores]
X_test2_full = X_test2_full[lst_melhores]
X_train2_full.fillna(0,inplace=True)
X_test2_full.fillna(0,inplace=True)
modelo3 = LogisticRegression(random_state=5,max_iter=1000000,C=3).fit(X_train2_full.drop(['dsm_mddh'],axis=1),y_train2)

In [74]:
score_treino3 = modelo3.score(X_train2_full.drop(['dsm_mddh'],axis=1),y_train2)

In [75]:
print('Score de Acuracia [TREINO]:',score_treino3)

Score de Acuracia [TREINO]: 0.964780252859723


In [76]:
score_teste3 = modelo3.score(X_test2_full.drop(['dsm_mddh'],axis=1),y_test2)

In [77]:
print('Score de Acuracia [TESTE]:',score_teste3)

Score de Acuracia [TESTE]: 0.9602888086642599


In [78]:
cont = 0
for i in lst_melhores:
    if i in Lista_Melhores2:
        cont = cont + 1
print(cont)

51


In [79]:
y_pred2 = modelo2.predict(X_test2.drop(['dsm_mddh'],axis=1))


In [80]:
print(classification_report(y_test2,y_pred2))

              precision    recall  f1-score   support

           1       0.94      0.98      0.96       831
           5       0.98      0.94      0.96       831

    accuracy                           0.96      1662
   macro avg       0.96      0.96      0.96      1662
weighted avg       0.96      0.96      0.96      1662



![Precision](https://i.imgur.com/lBX6xb0.png)
![Recall](https://i.imgur.com/auzyA0v.png)
![F1-Score](https://i.imgur.com/uRnhSGP.png)


* #### f1-score = Média harmônica de Precision e Recall